In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

In [3]:
from pyspark.sql import Row

In [4]:
spark = SparkSession.builder.getOrCreate()

22/10/29 10:49:03 WARN Utils: Your hostname, wangjiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.249.116.154 instead (on interface en0)
22/10/29 10:49:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/29 10:49:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import findspark
findspark.init()

In [6]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.15.0 pyspark-shell'

In [7]:
from pyspark import find_spark_home
print(find_spark_home._find_spark_home())

/Users/wangjiangyi/miniforge3/lib/python3.8/site-packages/pyspark


# Import MAS DB

In [6]:
df_author = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", "author") \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_writes = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", "writes") \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [8]:
df_publication = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", "publication") \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [9]:
df_author.printSchema()

root
 |-- aid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- oid: integer (nullable = true)
 |-- homepage: string (nullable = true)
 |-- photo: string (nullable = true)



In [10]:
df_writes.printSchema()

root
 |-- aid: integer (nullable = true)
 |-- pid: integer (nullable = true)



In [11]:
df_publication.printSchema()

root
 |-- pid: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- cid: integer (nullable = true)
 |-- jid: integer (nullable = true)
 |-- reference_num: integer (nullable = true)
 |-- citation_num: integer (nullable = true)
 |-- doi: string (nullable = true)



# Import DBLP DB

In [19]:
# define schema
article_schema = StructType([
    StructField("author", ArrayType(elementType = StringType()), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True)])

In [20]:
df_article = spark.read \
     .format('xml') \
     .option('rootTag', 'dblp') \
     .option('rowTag', 'article') \
     .load("dblp.xml", schema = article_schema)

In [21]:
df_article.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [34]:
df_article.show(5)

+--------------------+--------------------+----+
|              author|               title|year|
+--------------------+--------------------+----+
|[Paul Kocher, Dan...|Spectre Attacks: ...|2018|
|[Moritz Lipp, Mic...|            Meltdown|2018|
|      [Frank Manola]|An Evaluation of ...|1994|
|[Michael L. Brodi...|DARWIN: On the In...|1993|
|[Mark F. Hornick,...|Integrating Heter...|1991|
+--------------------+--------------------+----+
only showing top 5 rows



In [28]:
# define schema
inproc_schema = StructType([
    StructField("author", ArrayType(elementType = StringType()), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True)])

In [30]:
df_inproc = spark.read \
     .format('xml') \
     .option('rootTag', 'dblp') \
     .option('rowTag', 'inproceedings') \
     .load("dblp.xml", schema = inproc_schema)

In [31]:
df_inproc.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [33]:
df_inproc.show(5)

+--------------------+--------------------+----+
|              author|               title|year|
+--------------------+--------------------+----+
|   [Arnon Rosenthal]|The Future of Cla...|1998|
|        [E. F. Codd]|Seven Steps to Re...|1974|
|[Toni Bollinger, ...|The LILOG Inferen...|1991|
|[Kai-Uwe Carstens...|Why a Hill Can't ...|1991|
|[David W. Flater,...|Towards Flexible ...|1993|
+--------------------+--------------------+----+
only showing top 5 rows



# Query 1:

## 1. Max_year in MAS DB
- when I run spark.sql, it will always suffer from time limit issues
- so I can only pre-process the data and load to Jupyter Notebook

- this cannot work!!!

In [12]:
df_author.createOrReplaceTempView('author')
df_writes.createOrReplaceTempView('writes')
df_publication.createOrReplaceTempView('publication')

In [16]:
maxyear_MAS = spark.sql("select max(year) as max_year from publication\
                    where pid in \
                    (select pid from writes \
                    where aid = (select aid from author where name = 'Divesh Srivastava'))")

- this works

In [35]:
query_maxyear = "(select max(year) as max_year from publication\
                where pid in \
                (select pid from writes \
                where aid = (select aid from author where name = 'Divesh Srivastava'))\
                ) tmp"

In [36]:
maxyear_MAS = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", query_maxyear) \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [37]:
maxyear_MAS.show()

+--------+
|max_year|
+--------+
|    2012|
+--------+



In [43]:
max_year = maxyear_MAS.collect()

In [52]:
max_year[0]['max_year']

2012

## 2. Find all papers from DBLP that are published after 'Max_year' and written by 'Divesh S.'

In [55]:
from pyspark.sql.functions import array_contains

In [195]:
article_DS = df_article.filter((df_article.year > max_year[0]['max_year']) & (array_contains(df_article.author, 'Divesh Srivastava')))

In [196]:
article_DS.show()

+--------------------+--------------------+----+
|              author|               title|year|
+--------------------+--------------------+----+
|[Leon Bornemann, ...|Data Change Explo...|2018|
|[Philip E. Brown,...|From Rocks to Peb...|2019|
|[Donatella Firman...|Robust Entity Res...|2018|
|[Tamraparni Dasu,...|Data Quality for ...|2016|
|[Luciano Barbosa,...|Big Data Integrat...|2018|
|[Shazia W. Sadiq,...|Letter from the S...|2016|
|[Kareem El Gebaly...| Explanation Tables.|2018|
| [Divesh Srivastava]|Letter from the I...|2021|
|[Divesh Srivastav...|Ensuring High-Qua...|2019|
|[Aline Bessa, Jul...|Effective Discove...|2020|
|[Abolfazl Asudeh,...|Efficient Signal ...|2019|
|[Shazia Wasim Sad...|Data Quality: The...|2017|
|[Mehdi Kargar, Lu...|Effective Keyword...|2022|
|[Graham Cormode, ...|Constrained Priva...|2021|
|[Lukasz Golab, Ho...|Discovering Conse...|2014|
|[Hongyun Cai 0001...|Indexing Evolving...|2015|
|[Sainyam Galhotra...|Efficient and Eff...|2020|
|[Valter Crescenzi..

In [197]:
inproc_DS = df_inproc.filter((df_inproc.year > max_year[0]['max_year']) & (array_contains(df_inproc.author, 'Divesh Srivastava')))

In [198]:
inproc_DS.show()

+--------------------+--------------------+----+
|              author|               title|year|
+--------------------+--------------------+----+
|[Graham Cormode, ...|UMicS: from anony...|2013|
|[Divesh Srivastav...|Exploring and Ana...|2022|
|[Xi He 0001, Ashw...|Composing Differe...|2017|
|[Philip E. Brown,...|     Don't Cry Wolf.|2019|
|[Mehdi Kargar, Lu...|Effective Keyword...|2021|
|[Xian Li, Xin Lun...|Scaling up copy d...|2015|
|[Alexandar Mihayl...|FASTOD: Bringing ...|2018|
|[Trong Duc Nguyen...|Random Sampling f...|2020|
|[Alexandra Kim, L...|Summarizing Hiera...|2020|
|[Guoyao Feng, Luk...|Scalable Informat...|2017|
|[Leon Bornemann, ...|Structured Object...|2021|
|[Xin Luna Dong, D...|Big data integrat...|2013|
|[Hongyun Cai 0001...|Indexing evolving...|2016|
|[Pei Li, Jessica ...|abcOD: Mining Ban...|2022|
|[Grigory Yaroslav...|Accurate and effi...|2013|
|[Lukasz Golab, Fl...|Size-Constrained ...|2015|
|[Siarhei Bykau, F...|Fine-grained cont...|2015|
|[Graham Cormode, ..

In [250]:
query1_result = article_DS.select('title', 'year').union(inproc_DS.select('title', 'year')).dropDuplicates()

In [251]:
query1_result.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|From Rocks to Peb...|2019|
|Data Change Explo...|2018|
|Data Quality for ...|2016|
|Big Data Integrat...|2018|
|Letter from the I...|2021|
|Letter from the S...|2016|
|Robust Entity Res...|2018|
| Explanation Tables.|2018|
|Ensuring High-Qua...|2019|
|Effective Discove...|2020|
|Data Quality: The...|2017|
|Efficient Signal ...|2019|
|Effective Keyword...|2022|
|Constrained Priva...|2021|
|Indexing Evolving...|2015|
|Discovering Conse...|2014|
|Truth Finding on ...|2015|
|Alaska: A Flexibl...|2021|
|Errata Note: Disc...|2019|
|Efficient and Eff...|2020|
+--------------------+----+
only showing top 20 rows



# Query 2:

## 1. convert all data into author-title form
- those authors who collobrate with Diversh S.

- mysql

In [77]:
query_coop_MAS = "(select b.name as author, c.title as title from \
                    ( \
                        select aid, pid from writes \
                        where pid in (select pid from writes where aid = (select aid from author where name = 'Divesh Srivastava')) \
                            and aid != (select aid from author where name = 'Divesh Srivastava') \
                    ) a \
                left join \
                (select aid, name from author) b \
                on a.aid = b.aid \
                left join \
                (select pid, title from publication) c \
                on a .pid = c.pid \
                    ) tmp"

In [78]:
coop_MAS = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", query_coop_MAS) \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [105]:
coop_MAS.show()

+--------------------+--------------------+
|              author|               title|
+--------------------+--------------------+
|      Mary Fernandez|PIX: exact and ap...|
|            Y. L. Xu|PIX: exact and ap...|
|    Sihem Amer-yahia|PIX: exact and ap...|
|     S Muthukrishnan|Using q-grams in ...|
|Panagiotis G. Ipe...|Using q-grams in ...|
|        Luis Gravano|Using q-grams in ...|
|      H. V. Jagadish|Using q-grams in ...|
|         Nick Koudas|Using q-grams in ...|
|    Lauri Pietarinen|Using q-grams in ...|
|Panagiotis G. Ipe...|Text Joins for Da...|
|        Luis Gravano|Text Joins for Da...|
|         Nick Koudas|Text Joins for Da...|
|       Nicolas Bruno|Navigation vs. In...|
|        Luis Gravano|Navigation vs. In...|
|         Nick Koudas|Navigation vs. In...|
|Panagiotis G. Ipe...|Text joins in an ...|
|        Luis Gravano|Text joins in an ...|
|         Nick Koudas|Text joins in an ...|
|      Yannis Kotidis| Ranked Join Indices|
| Panayiotis Tsaparas| Ranked Jo

In [209]:
coop_MAS.first()

Row(author='Mary Fernandez', title='PIX: exact and approximate phrase matching in XML')

- it can be observed that:
    - in MAS, article name **does not** include '.';
    - in dblp, article name includes '.';
- modification should be made to MAS data

In [143]:
def modify(tup):
    author = tup[0]
    title = tup[1]
    return {'author': author, 'title': title + '.'}
coop_MAS_modify = coop_MAS.rdd.map(modify).toDF()

In [145]:
coop_MAS_modify.first()

Row(author='Mary Fernandez', title='PIX: exact and approximate phrase matching in XML.')

In [172]:
coop_MAS_modify.show()

+--------------------+--------------------+
|              author|               title|
+--------------------+--------------------+
|      Mary Fernandez|PIX: exact and ap...|
|            Y. L. Xu|PIX: exact and ap...|
|    Sihem Amer-yahia|PIX: exact and ap...|
|     S Muthukrishnan|Using q-grams in ...|
|Panagiotis G. Ipe...|Using q-grams in ...|
|        Luis Gravano|Using q-grams in ...|
|      H. V. Jagadish|Using q-grams in ...|
|         Nick Koudas|Using q-grams in ...|
|    Lauri Pietarinen|Using q-grams in ...|
|Panagiotis G. Ipe...|Text Joins for Da...|
|        Luis Gravano|Text Joins for Da...|
|         Nick Koudas|Text Joins for Da...|
|       Nicolas Bruno|Navigation vs. In...|
|        Luis Gravano|Navigation vs. In...|
|         Nick Koudas|Navigation vs. In...|
|Panagiotis G. Ipe...|Text joins in an ...|
|        Luis Gravano|Text joins in an ...|
|         Nick Koudas|Text joins in an ...|
|      Yannis Kotidis|Ranked Join Indices.|
| Panayiotis Tsaparas|Ranked Joi

- dblp

In [150]:
from pyspark.sql.functions import explode

In [212]:
df_article_update = df_article.filter(array_contains(df_article.author, 'Divesh Srivastava')).select(explode('author').alias('author'), 'title')

In [174]:
df_article_coop = df_article_update.filter(df_article_update.author != 'Divesh Srivastava')
df_article_coop.show()

+--------------------+--------------------+
|              author|               title|
+--------------------+--------------------+
|      Leon Bornemann|Data Change Explo...|
|Dmitri V. Kalashn...|Data Change Explo...|
|       Felix Naumann|Data Change Explo...|
|  Raghu Ramakrishnan|Pushing Constrain...|
|     Philip E. Brown|From Rocks to Peb...|
|     Tamraparni Dasu|From Rocks to Peb...|
|         Yaron Kanza|From Rocks to Peb...|
|   Donatella Firmani|Robust Entity Res...|
|    Sainyam Galhotra|Robust Entity Res...|
|          Barna Saha|Robust Entity Res...|
|        Luis Gravano|Using q-grams in ...|
|Panagiotis G. Ipe...|Using q-grams in ...|
|      H. V. Jagadish|Using q-grams in ...|
|         Nick Koudas|Using q-grams in ...|
|S. Muthukrishnan ...|Using q-grams in ...|
|    Lauri Pietarinen|Using q-grams in ...|
|     Tamraparni Dasu|Data Quality for ...|
|           Rong Duan|Data Quality for ...|
|        Lukasz Golab|Efficient and Eff...|
|           Flip Korn|Efficient 

In [170]:
df_inproc_update = df_inproc.filter(array_contains(df_inproc.author, 'Divesh Srivastava')).select(explode('author').alias('author'), 'title')

In [173]:
df_inproc_coop = df_inproc_update.filter(df_inproc_update.author != 'Divesh Srivastava')
df_inproc_coop.show()

+--------------------+--------------------+
|              author|               title|
+--------------------+--------------------+
|  Raghu Ramakrishnan|Efficient Increme...|
|     Kenneth A. Ross|Efficient Increme...|
|   S. Sudarshan 0001|Efficient Increme...|
|       David B. Kemp|Magic Sets and Bo...|
|    Peter J. Stuckey|Magic Sets and Bo...|
|   S. Sudarshan 0001|Extending the Wel...|
|  Raghu Ramakrishnan|Extending the Wel...|
|       Catriel Beeri|Extending the Wel...|
|         Nick Koudas|Data Stream Query...|
|      H. V. Jagadish|Flexible List Man...|
|       Mark A. Jones|Flexible List Man...|
|       Dimitra Vista|Flexible List Man...|
|        Guy Jacobson|Focusing Search i...|
|Balachander Krish...|Focusing Search i...|
|           Dan Suciu|Focusing Search i...|
|      Graham Cormode|UMicS: from anony...|
|         Entong Shen|UMicS: from anony...|
|             Xi Gong|UMicS: from anony...|
|        Ting Yu 0001|UMicS: from anony...|
|Cecilia M. Procopiuc|UMicS: fro

## 2. union 3 dataframes and drop duplicates

In [241]:
union_coop = coop_MAS_modify.union(df_article_coop).union(df_inproc_coop).dropDuplicates()

In [242]:
union_coop.show()

+--------------------+--------------------+
|              author|               title|
+--------------------+--------------------+
|Suresh Venkatasub...|Type-based catego...|
|Yannis Papakonsta...|Keyword proximity...|
|        S. Paparizos|TIMBER: A Native ...|
|         Nick Koudas|Metric Functional...|
|      H. V. Jagadish|Snakes and sandwi...|
|  Raghu Ramakrishnan|Implementation of...|
|     S Muthukrishnan|Efficient Approxi...|
|        Kenneth Ross|Foundations of Ag...|
|   Michael O. Akinde|Efficient OLAP Qu...|
|         Nick Koudas|Multiple aggregat...|
|         Songtao Guo|Record Linkage wi...|
|        S. Sudarshun|The coral user ma...|
|      Yannis Kotidis|Updates Through V...|
|          Qing Zhang|Aggregate Query A...|
|       S. Sudarshant|Cost-based optimi...|
|     Ugur Cetintemel|Evaluating Answer...|
|       Andrey Balmin|A System for Keyw...|
|     Anand Deshpande|Proceedings of th...|
|          Yu-Qing Wu|    Grouping in XML.|
|        S. Sudarshan|Foundation

## 3. Groupby + Count

In [243]:
tmp = union_coop.select('author').groupby('author').count()

In [244]:
query2_result = tmp.orderBy(tmp['count'].desc()).limit(1)

In [245]:
query2_result.show()

+-----------+-----+
|     author|count|
+-----------+-----+
|Nick Koudas|   86|
+-----------+-----+



# Query 3:

## 1. convert all data into title-year form
- those paper whose author contains 'Divesh S.'

- mysql

In [202]:
query_DS_paper_MAS = "(select \
                        b.title as title, \
                        b.year 			as year \
                    from \
                        ( \
                    select pid from writes where aid = (select aid from author where name = 'Divesh Srivastava') \
                        ) a \
                    left join publication b \
                    on a.pid = b.pid) tmp"

In [203]:
DS_paper_MAS = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", query_DS_paper_MAS) \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [204]:
DS_paper_MAS.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|PIX: exact and ap...|2003|
|Using q-grams in ...|2001|
|Text Joins for Da...|2003|
|Navigation vs. In...|2003|
|Text joins in an ...|2003|
| Ranked Join Indices|2003|
|TIMBER: a native ...|2003|
|     Grouping in XML|2002|
|Optimizing the se...|2002|
|TIMBER: A native ...|2002|
|TAX: A Tree Algeb...|2001|
|Structural Joins:...|2002|
|On Bounding-Schem...|2000|
|Minimization of T...|2001|
|One-dimensional a...|2000|
|Approximate Strin...|2001|
|Counting Twig Mat...|2001|
|Coral++: Adding O...|1993|
|Implementation of...|1993|
|Rule Ordering in ...|1990|
+--------------------+----+
only showing top 20 rows



In [206]:
DS_paper_MAS.first()

Row(title='PIX: exact and approximate phrase matching in XML', year=2003)

- it can be observed that: (**same as query 2**)
    - in MAS, article name **does not** include '.';
    - in dblp, article name includes '.';
- modification should be made to MAS data

In [205]:
def modify_paper(tup):
    title = tup[0]
    year = tup[1]
    return {'title': title + '.', 'year': year}
DS_paper_MAS_modify = DS_paper_MAS.rdd.map(modify_paper).toDF()

In [207]:
DS_paper_MAS_modify.first()

Row(title='PIX: exact and approximate phrase matching in XML.', year=2003)

In [208]:
DS_paper_MAS_modify.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|PIX: exact and ap...|2003|
|Using q-grams in ...|2001|
|Text Joins for Da...|2003|
|Navigation vs. In...|2003|
|Text joins in an ...|2003|
|Ranked Join Indices.|2003|
|TIMBER: a native ...|2003|
|    Grouping in XML.|2002|
|Optimizing the se...|2002|
|TIMBER: A native ...|2002|
|TAX: A Tree Algeb...|2001|
|Structural Joins:...|2002|
|On Bounding-Schem...|2000|
|Minimization of T...|2001|
|One-dimensional a...|2000|
|Approximate Strin...|2001|
|Counting Twig Mat...|2001|
|Coral++: Adding O...|1993|
|Implementation of...|1993|
|Rule Ordering in ...|1990|
+--------------------+----+
only showing top 20 rows



- dblp

In [214]:
df_article_paper = df_article.filter(array_contains(df_article.author, 'Divesh Srivastava')).select('title', 'year')

In [215]:
df_article_paper.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|Data Change Explo...|2018|
|Pushing Constrain...|1993|
|From Rocks to Peb...|2019|
|Robust Entity Res...|2018|
|Using q-grams in ...|2001|
|Data Quality for ...|2016|
|Efficient and Eff...|2011|
|Big Data Integrat...|2018|
|Letter from the S...|2016|
| Explanation Tables.|2018|
|Letter from the I...|2021|
|Semantics and Opt...|1994|
|Weighted Set-Base...|2010|
|Finding hierarchi...|2008|
|Approximate Strin...|2011|
|Ensuring High-Qua...|2019|
|Generating Advanc...|1998|
|Finding the K hig...|2009|
|Special issue: be...|2012|
|Iterative multi-t...|2003|
+--------------------+----+
only showing top 20 rows



In [217]:
df_inproc_paper = df_inproc.filter(array_contains(df_inproc.author, 'Divesh Srivastava')).select('title', 'year')

In [218]:
df_inproc_paper.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|Efficient Increme...|1994|
|Magic Sets and Bo...|1991|
|Extending the Wel...|1993|
|Data Stream Query...|2003|
|Flexible List Man...|1998|
|Focusing Search i...|1998|
|UMicS: from anony...|2013|
|Logical and physi...|2002|
|  Schema extraction.|2010|
|Exploring and Ana...|2022|
|Composing Differe...|2017|
|A mapping schema ...|2002|
|Rapid Identificat...|2006|
|08111 Abstracts C...|2008|
|08111 Report - Ra...|2008|
|Explaining Progra...|1993|
|     Don't Cry Wolf.|2019|
|Effective Computa...|2005|
|Metric Functional...|2009|
|Efficient Process...|2008|
+--------------------+----+
only showing top 20 rows



## 2. union 3 dataframes and drop duplicates

In [238]:
union_paper = DS_paper_MAS_modify.union(df_article_paper).union(df_inproc_paper).dropDuplicates()

In [246]:
union_paper.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|Pushing constrain...|1992|
|Differentially Pr...|2011|
|The Bellman Data ...|2006|
|On Bounding-Schem...|2000|
|The coral user ma...|1993|
|TIMBER: A Native ...|2003|
|Navigation vs. In...|2003|
|On Computing Corr...|2001|
|Review - Querying...|2000|
|Complex Aggregati...|1998|
|Approximate Strin...|2003|
|Anonymized Data: ...|2010|
|The calculation o...|1990|
|Aggregate Query A...|2012|
|Snakes and sandwi...|1999|
|Aggregate Query A...|   0|
|Constraint Databa...|1997|
|On Effective Mult...|2000|
|Implementation of...|1993|
|Efficient OLAP qu...|2002|
+--------------------+----+
only showing top 20 rows



## 3. Groupby + Count

In [252]:
query3_result = union_paper.groupby('year').count()

In [253]:
query3_result.show()

+----+-----+
|year|count|
+----+-----+
|   0|   10|
|2007|   15|
|2014|   10|
|2021|   16|
|2012|   18|
|1991|    3|
|2022|   16|
|2016|    8|
|null|    1|
|1994|    9|
|2018|   19|
|1999|   15|
|1997|    3|
|2010|   23|
|2009|   28|
|2006|   16|
|2017|   16|
|1998|   10|
|2013|   12|
|2004|   12|
+----+-----+
only showing top 20 rows



# Query 4: